#### Qwen3のファインチューニング
NVIDIA GeForce RTX 2070 SUPER(VRAM 8GB)にて、Qwen3-4Bに対してLoRAファインチューニングを実施。Qwen3-8Bは7時間弱かかるため1個下のモデルで実施。
Windows、GPUの古さが影響して、Tritonが利用できなかったため、これを利用しないように、インストールや環境変数は手を入れている。

[Unsloth公式ドキュメント](https://docs.unsloth.ai/)

※Colabローカルランタイムでの使用
1. ローカル環境で下記実行
   
   ```jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0 --NotebookApp.allow_credentials=True```
2. 表示されたトークンをColabに入力

#### ライブラリインストールとインポート

In [1]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    print("ローカル環境")
    !pip install unsloth
    !pip uninstall -y cut_cross_entropy # Windows環境だとうまく動かないため。
    os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"        # TorchDynamo / Triton を使わない(Windows環境だとうまく動かないため。)
    os.environ["UNSLOTH_COMPILE_IGNORE_ERRORS"] = "1"  # エラーが出ても fallback
    DATASE_NUM_PROC = 1
else:
    print("Google Colab環境")
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    DATASE_NUM_PROC = None
    

ローカル環境


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

from transformers import TextIteratorStreamer
from threading import Thread
import textwrap


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!


#### Unsloth
モデルとトークナイザーの読み込み

In [3]:
# %env UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT

In [4]:
# モデル一覧（公式ドキュメントとHuggingFace）
# https://docs.unsloth.ai/get-started/all-our-models
# https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

c:\Users\tmina\anaconda3\envs\myenv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.4.8: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 2070 SUPER. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LoRA adaptersを追加

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.4.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


<a id="Data"></a>
### データ準備
Qwen3には推論モードと非推論モードがあり、非推論のデータセットだけだと推論能力に影響を与える可能性があるため、双方のデータセットを組み合わせる形でデータ構築:

1. We use the [Open Math Reasoning]() dataset which was used to win the [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) challenge! We sample 10% of verifiable reasoning traces that used DeepSeek R1, and whicht got > 95% accuracy.

2. We also leverage [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we need to convert it to HuggingFace's normal multiturn format as well.
   
##### 金融データ
- https://huggingface.co/datasets/FinGPT/fingpt-sentiment-train
- https://huggingface.co/datasets/FinGPT/fingpt-fiqa_qa
- https://github.com/czyssrs/FinQA

In [47]:
# dataset = load_dataset("roneneldan/TinyStories", split = "train[:2500]")
dataset = load_dataset('text', data_files="output.txt", split = "train")
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
dataset = dataset.map(formatting_prompts_func, batched = True,)

for row in dataset[:5]["text"]:
    print("=========================")
    print(row)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

# 主要行等向けの総合的な監督指針 令和７年６月## I 基本的考え方## II 主要行等の検査・監督に係る事務処理上の留意点- II －１ 検査・監督事務に係る基本的考え方- II －１－１ 検査・監督事務の進め方- II －１－２ 検査・監督事務の具体的手法- II －１－３ 品質管理- II －１－４ 財務局との連携- II －１－５ 預金保険機構が行う検査との連携- II －１－６ 個別銀行に関する行政報告等- II －１－７ 銀行等が提出する申請書等における記載上の留意点- II －１－８ 書面・対面による手続きについての留意点- II －１－９ 申請書等を提出するに当たっての留意点- II －２ 銀行に関する苦情・情報提供等- II －２－１ 相談・苦情等を受けた場合の対応- II －２－２ 金融サービス利用者相談室との連携- II －２－３ 金融サービス利用者相談室で受け付けた情報のうち、いわゆる貸し渋り・貸し剥がしとして提供された情報に係る監督上の対応- II －２－４ 預金口座を利用した架空請求等預金口座の不正利用に関する情報を受けた場合の対応- II －３ 法令解釈等の照会を受けた場合の対応- II －３－１ 照会を受ける内容の範囲- II －３－２ 照会に対する回答方法- II －３－３ 法令適用事前確認手続（ノーアクションレター制度）- II －３－４ グレーゾーン解消制度- II －３－５ 預金等に対する当局への照会等への対応- II －４ 行政指導等を行う際の留意点等- II －４－１ 行政指導等を行う際の留意点- II －４－２ 面談等を行う際の留意点- II －５ 行政処分等を行う際の留意点等- II －５－１ 行政処分（不利益処分）に関する基本的な事務の流れについて- II －５－１－１ 行政処分- II －５－１－２ 法第26条に基づく業務改善命令の履行状況の報告義務の解除- II －５－２ 行政手続法との関係等- II －５－３ 意見交換制度- II －５－４ 関係当局・海外監督当局等への連絡- II －５－５ 不利益処分の公表に関する考え方- II －５－６ 予備審査- II －５－１ 行政処分（不利益処分）に関する基本的な事務の流れについて## III 主要行等監督上の評価項目- III －１ 経営管理（ガバナンス）- II

In [48]:
# テキストを連結し、またトークン数の合計を計算する
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")
combined_text = "".join(dataset["text"])
total_tokens = enc.encode(combined_text)
# 結果を表示
print(f"Total texts: {len(combined_text)}")
print(f"Total tokens: {len(total_tokens)}")

Total texts: 2906692
Total tokens: 2418679


In [49]:
print(combined_text[:1000])

# 主要行等向けの総合的な監督指針 令和７年６月## I 基本的考え方## II 主要行等の検査・監督に係る事務処理上の留意点- II －１ 検査・監督事務に係る基本的考え方- II －１－１ 検査・監督事務の進め方- II －１－２ 検査・監督事務の具体的手法- II －１－３ 品質管理- II －１－４ 財務局との連携- II －１－５ 預金保険機構が行う検査との連携- II －１－６ 個別銀行に関する行政報告等- II －１－７ 銀行等が提出する申請書等における記載上の留意点- II －１－８ 書面・対面による手続きについての留意点- II －１－９ 申請書等を提出するに当たっての留意点- II －２ 銀行に関する苦情・情報提供等- II －２－１ 相談・苦情等を受けた場合の対応- II －２－２ 金融サービス利用者相談室との連携- II －２－３ 金融サービス利用者相談室で受け付けた情報のうち、いわゆる貸し渋り・貸し剥がしとして提供された情報に係る監督上の対応- II －２－４ 預金口座を利用した架空請求等預金口座の不正利用に関する情報を受けた場合の対応- II －３ 法令解釈等の照会を受けた場合の対応- II －３－１ 照会を受ける内容の範囲- II －３－２ 照会に対する回答方法- II －３－３ 法令適用事前確認手続（ノーアクションレター制度）- II －３－４ グレーゾーン解消制度- II －３－５ 預金等に対する当局への照会等への対応- II －４ 行政指導等を行う際の留意点等- II －４－１ 行政指導等を行う際の留意点- II －４－２ 面談等を行う際の留意点- II －５ 行政処分等を行う際の留意点等- II －５－１ 行政処分（不利益処分）に関する基本的な事務の流れについて- II －５－１－１ 行政処分- II －５－１－２ 法第26条に基づく業務改善命令の履行状況の報告義務の解除- II －５－２ 行政手続法との関係等- II －５－３ 意見交換制度- II －５－４ 関係当局・海外監督当局等への連絡- II －５－５ 不利益処分の公表に関する考え方- II －５－６ 予備審査- II －５－１ 行政処分（不利益処分）に関する基本的な事務の流れについて## III 主要行等監督上の評価項目- III －１ 経営管理（ガバナンス）- II

<a name="Train"></a>
### 学習
Huggingface TRLの`SFTTrainer`を使用。[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)

60ステップを実行しているが、`num_train_epochs=1`を設定してフルに実行し、`max_steps=None`をオフにすることも可

In [50]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = DATASE_NUM_PROC,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/39 [00:00<?, ? examples/s]

In [51]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2070 SUPER. Max memory = 8.0 GB.
7.189 GB of memory reserved.


※学習を再開するには、`trainer.train(resume_from_checkpoint = True)`を設定

In [52]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 39 | Num Epochs = 1 | Total steps = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 843,972,608/4,000,000,000 (21.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.986500
2,1.844500


In [53]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1647.9157 seconds used for training.
27.47 minutes used for training.
Peak reserved memory = 19.74 GB.
Peak reserved memory for training = 12.551 GB.
Peak reserved memory % of max memory = 246.75 %.
Peak reserved memory for training % of max memory = 156.887 %.


<a name="Inference"></a>
### 推論
Qwen-3チームによると、推論に推奨される設定は `temperature = 0.6, top_p = 0.95, top_k = 20` で、

通常のチャットベースの推論では、`temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

# Before running inference, call `FastLanguageModel.for_inference` first

FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    "金融商品取引法とは、"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

Financial Instruments and Exchange Act is
ある種の金融商品取引に関する法律であり、金融商品取引業者の登録、開示義務、金融商品取引所の設立などを規定している。金融商品取引業者は、顧客に対して適切な情報提供を行い、公正な取引を確保するための措置を講じる必要がある。また、金融商品取引所は、上場企業に対して適切な開示を求めるとともに、投資家保護のための監視機能を果たすことが求められる。この. The 

Exception in thread Thread-29 (generate):
Traceback (most recent call last):
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-30 (generate):
Traceback (most recent call last):
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\threading.py", line 975, in run
    self.run()
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\site-packages\peft\peft_model.py", line 1875, in generate
    self._target(*self._args, **self._kwargs)
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\site-packages\peft\peft_model.py", line 1875, in generate
    outputs = self.base_model.generate(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tmina\anaconda3\envs\myenv\Lib\site

<a name="Save"></a>
### モデルの保存と読み込み
最終モデルをLoRAアダプターとして保存するには、オンライン保存の場合はHuggingfaceの`push_to_hub`を、ローカル保存の場合は`save_pretrained`を使う

**[NOTE]** これはLoRAアダプターだけを保存し、フルモデルは保存しません。16bitまたはGGUFに保存するには、最下部参照

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model\\tokenizer_config.json',
 'lora_model\\special_tokens_map.json',
 'lora_model\\vocab.json',
 'lora_model\\merges.txt',
 'lora_model\\added_tokens.json',
 'lora_model\\tokenizer.json')

推論用に保存した LoRA アダプターをロードしたい場合は、`False` を `True` に設定する

In [23]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### VLLMのためのfloat16への保存
float16 の場合は `merged_16bit` を、int4 の場合は `merged_4bit` を選択

push_to_hub_merged`を使用してHugging Faceアカウントにアップロード可（要トークン）

In [24]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp 変換
`GGUF` / `llama.cpp` への保存は現在ネイティブでサポート。`llama.cpp`をクローンし、デフォルトで `q8_0` に保存。`q4_k_m` のようなすべてのメソッドを許可する。ローカルに保存する場合は `save_pretrained_gguf` を、HF にアップロードする場合は `push_to_hub_gguf` を使用する。

サポートされているクォンツメソッド (完全なリストは[Wikiページ](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options))：
* `q8_0` - 高速変換。リソースの使用量は多いが、一般的には許容範囲。
* `q4_k_m` - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使い、それ以外はQ4_Kを使う。
* `q5_k_m` - 推奨。attention.wvとfeed_forward.w2のテンソルの半分にQ6_Kを使用、それ以外はQ5_K。

[Ollama ノートブック](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)をお試しください。

In [25]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

llama.cppの`model.gguf`ファイルか`model-Q4_K_M.gguf`ファイル、またはJanやOpen WebUIなどのUIベースのシステムを使用してください。Jan [こちら](https://github.com/janhq/jan) と Open WebUI [こちら](https://github.com/open-webui/open-webui) をインストールしてください。

これで完了です！Unslothについて質問があれば、[Discord](https://discord.gg/unsloth)チャンネルがあります！バグを見つけたり、LLMの最新情報を入手したり、ヘルプが必要な場合、プロジェクトに参加したい場合など、お気軽にDiscordに参加してください！

その他のリンク
1. 自分の推論モデルを訓練する - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Ollamaにfinetunesを保存する。[フリーノート](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [フリーColab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb) 6.
6. DPO, ORPO, Continued pre-training, conversational finetuningなどのノートブックは[ドキュメント](https://docs.unsloth.ai/get-started/unsloth-notebooks)をご覧ください！